In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from haversine import haversine
from sklearn.model_selection import train_test_split
np.random.seed(42)
import scipy.stats as stats
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from datetime import timedelta  


from matplotlib import rcParams
rcParams.update({'figure.autolayout':True})

plt.style.use(["presentation"])

%matplotlib inline

In [3]:
df1=pd.read_pickle('./res_parking_permit_violations')

In [4]:
df2=pd.read_csv('./Residential_Parking_Permit_Blocks.csv')

In [5]:
df1.columns

Index(['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION',
       'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT',
       'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE',
       'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME',
       'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY',
       'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY',
       'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD', 'DETAILS'],
      dtype='object')

In [6]:
features=['OBJECTID', 'SERVICECODEDESCRIPTION', 'ADDDATE', 'RESOLUTIONDATE', 'STREETADDRESS',
          'LATITUDE', 'LONGITUDE', 'ZIPCODE', 'WARD', 'DETAILS']

In [7]:
# Creating a smaller version of the dataframe with just the features that might be useful

df_short=df1[features]
df_short = df_short.copy()

In [8]:
# Converting dates to Pandas Date-Time Format

df_short['ADDDATE']=df_short['ADDDATE'].map(pd.to_datetime)
df_short['RESOLUTIONDATE']=df_short['RESOLUTIONDATE'].map(pd.to_datetime)


In [9]:
df_short['day_of_week']=df_short['ADDDATE'].map(lambda x : x.weekday_name)

In [10]:
df_short['day_of_week'].value_counts()

Friday       5982
Thursday     5830
Wednesday    5809
Tuesday      5643
Monday       4954
Saturday     1392
Sunday        197
Name: day_of_week, dtype: int64

In [11]:
# Month parking violation occurred

df_short['month']=df_short['ADDDATE'].map(lambda x : x.month)

In [12]:
# Time in minutes to resolve complaint

df_short['resolvetime']=(df_short['RESOLUTIONDATE']-df_short['ADDDATE']).astype('timedelta64[m]')

In [14]:
df_short.to_csv('./shorter.csv')

In [54]:
# Code from Ben



import re

def get_address_block(address):
    try:
        m = re.findall('(\d+\s)', address)
        return(np.mean([float(x.strip()) for x in m]))
    except:
        return(np.nan)

df_short['BLOCKNUM']=df_short['STREETADDRESS'].apply(get_address_block)

def round_address_block(address):
    try:
        address=address/100
        address=np.round(address,0)
        address=int(address*100)
        return address
    except:
        return(np.nan)

df_short['BLOCKNUM']=df_short['BLOCKNUM'].map(round_address_block)

/anaconda3/envs/dsi/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [65]:
def get_street_name(address):
    try:
        m=address.replace("BLOCK OF","")
        return m
    except:
        return (np.nan)

df_short['STREETADDRESS'].apply(get_street_name)

58                       7436 GEORGIA AVENUE NW
149                          5617 8TH STREET NW
227                    1200 - 1299  D STREET NE
283                         1619 LYMAN PLACE NE
445                      2716 CENTRAL AVENUE NE
489                         1619 LYMAN PLACE NE
656           1200 - 1299  LONGFELLOW STREET NW
683                    1500 - 1599  W STREET NE
723                 3100 - 3199  14TH STREET NE
725               1400 - 1499  IRVING STREET NE
728              4600 - 4699  QUARLES STREET NE
731                 4600 - 4615  POLK STREET NE
748                            20 46TH PLACE NE
798                    1700 - 1780  U STREET NW
871                       235 EMERSON STREET NW
997                          4831 BASS PLACE SE
1093             1400 - 1499  FLORIDA AVENUE NW
1118      4933 NANNIE HELEN BURROUGHS AVENUE NE
1137                      2827 NEWTON STREET NE
1145                        3205 POPE STREET SE
1155                        2919 10TH ST